In [9]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.auto import tqdm


In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/112.0'
}

In [5]:
class BaseCrauler:
    def __init__(self, base_url, url, max_pages = 1):
        self.base_url = base_url
        self.url = url
        self.soup = None
        self.queue = []
        self.max_pages = max_pages
        self.result = []
    
    def getPage(self, page):
        self.page = requests.get(self.url.format(page=page), headers=headers, 
                                 )
    
    def getSubPage(self, page):
        self.page = requests.get(f'{self.base_url}{page}', headers=headers)
    
    def getSoup(self):
        self.soup = BeautifulSoup(self.page.content, 'html.parser')
    
    def parseComments(self):
        return NotImplemented
    
    def proceed_queue(self):
        return NotImplemented
    
    def to_csv(self, file_name):
        df = pd.DataFrame(self.result)
        df.to_csv(file_name)

In [14]:
class TonkostiCrauler(BaseCrauler):
    def __init__(self, url, max_pages):
        super().__init__(base_url='https://tonkosti.ru',
                         url=url,
                         max_pages=max_pages)
    
    def parseComments(self, is_exact=False):
        try:
            parent = 'div' if is_exact else 'li'
            for card in self.soup.find_all(parent, class_='reviews__item'):
                    review_mark = card.find('div', class_="reviews__main-rating")
                    if review_mark:
                        review_mark = review_mark.find('span').text
                    else:
                        continue
                    
                    review_text_el = card.find('div', class_="reviews__texts")
                    
                    date_el = card.find('div', class_='reviews__date')
                    date = date_el.text
                    date_el.decompose()
                    
                    review_text = review_text_el.text
                    
                    if 'Читать отзыв целиком' in review_text:
                        try:
                            self.queue.append(card.find('div', class_='reviews__link-to-review-page').find('a').attrs['href'])
                        except:
                            print('Не смог найти ссылку')
                    else:
                        self.result.append({'mark': review_mark, 'review_text': review_text, 'date': date})
                        
        except Exception as e:
            print("Error on card", e)

    def _proceed_queue(self):
        for url in tqdm(self.queue):
            self.getSubPage(url)
            self.getSoup()
            self.parseComments(is_exact=True)
    
    def proceed_pages(self, pages=None):
        if pages is None: pages = self.max_pages
        
        print('Сбор комментариев')
        for page_number in tqdm(range(1, pages+1)):
            self.getPage(page_number)
            self.getSoup()
            self.parseComments()
        
        print("Сбор больших комментариев")
        self._proceed_queue()
        
        print(f'Done. Собрано {len(self.result)} комментария')
        
            

In [264]:
tonkosti = TonkostiCrauler(max_pages=169, url='https://tonkosti.ru/%D0%9E%D1%82%D0%B7%D1%8B%D0%B2%D1%8B_%D0%BE_%D0%9A%D1%80%D1%8B%D0%BC%D0%B5?page={page}')
tonkosti.proceed_pages()


Сбор комментариев


  0%|          | 0/169 [00:00<?, ?it/s]

Сбор больших комментариев


  0%|          | 0/849 [00:00<?, ?it/s]

got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got 

In [267]:
df = pd.DataFrame(tonkosti.result)

In [268]:
df

,mark,review_text,date
0,2,\nИП Боровик Ю.В. Ялта Всем здравствуйте! Спеш...,19 апреля 2023
1,10,\nПоход на Учан Су Были на водопаде 10 февраля...,10 февраля 2023
2,10,"\nздравствуйте,по крыму 17 лет отдыхаю,подходи...",26 декабря 2022
3,10,\nОдним словом - это отдых! Путешествовали по ...,11 ноября 2022
4,10,\nкрым -так выглядит русская душа. Всем доброг...,25 октября 2022
...,...,...,...
1300,6,"\nМыс Фиолент и Яшмовый пляж - красивое место,...",02 августа 2013
1301,9,\nМассандра – продолжение Ялты Массандра - мал...,02 августа 2013
1302,10,"\nШикарный город, но трудно поддающийся прогул...",02 августа 2013
1303,9,"\nАлушта, в которой отдыхаешь душой В Алуште я...",02 августа 2013


In [269]:
df.to_csv('result_tonkosti.csv')

In [272]:
tonkosti = TonkostiCrauler(max_pages=5, url='https://tonkosti.ru/%D0%9E%D1%82%D0%B7%D1%8B%D0%B2%D1%8B_%D0%BE_%D0%9A%D0%B8%D1%81%D0%BB%D0%BE%D0%B2%D0%BE%D0%B4%D1%81%D0%BA%D0%B5?page={page}')
tonkosti.proceed_pages()
tonkosti.to_csv('kislovods.csv')

Сбор комментариев


  0%|          | 0/5 [00:00<?, ?it/s]

Сбор больших комментариев


  0%|          | 0/19 [00:00<?, ?it/s]

got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
Done. Собрано 36 комментария


In [274]:
tonkosti = TonkostiCrauler(max_pages=26, url='https://tonkosti.ru/%D0%9E%D1%82%D0%B7%D1%8B%D0%B2%D1%8B_%D0%BE_%D0%93%D0%B5%D0%BB%D0%B5%D0%BD%D0%B4%D0%B6%D0%B8%D0%BA%D0%B5?page={page}')
tonkosti.proceed_pages()
tonkosti.to_csv('gelendgik.csv')

Сбор комментариев


  0%|          | 0/26 [00:00<?, ?it/s]

Сбор больших комментариев


  0%|          | 0/93 [00:00<?, ?it/s]

got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got subpage
got 

In [277]:

tonkosti = TonkostiCrauler(max_pages=26, url='https://tonkosti.ru/%D0%9E%D1%82%D0%B7%D1%8B%D0%B2%D1%8B_%D0%BE_%D0%A1%D0%BE%D1%87%D0%B8?page={page}')
tonkosti.proceed_pages()
tonkosti.to_csv('gelendgik.csv')

Сбор комментариев


  0%|          | 0/26 [00:00<?, ?it/s]

Сбор больших комментариев


  0%|          | 0/78 [00:00<?, ?it/s]

Done. Собрано 179 комментария


In [278]:

tonkosti = TonkostiCrauler(max_pages=39, url='https://tonkosti.ru/%D0%9E%D1%82%D0%B7%D1%8B%D0%B2%D1%8B_%D0%BE_%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B5?page={page}')
tonkosti.proceed_pages()
tonkosti.to_csv('moscow.csv')

Сбор комментариев


  0%|          | 0/39 [00:00<?, ?it/s]

Сбор больших комментариев


  0%|          | 0/180 [00:00<?, ?it/s]

Done. Собрано 333 комментария


In [292]:

tonkosti = TonkostiCrauler(max_pages=12, url='https://tonkosti.ru/%D0%9E%D1%82%D0%B7%D1%8B%D0%B2%D1%8B_%D0%BE_%D0%9A%D0%B0%D0%B2%D0%BA%D0%B0%D0%B7%D1%81%D0%BA%D0%B8%D1%85_%D0%9C%D0%B8%D0%BD%D0%B5%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D1%85_%D0%92%D0%BE%D0%B4%D0%B0%D1%85?page={page}')
tonkosti.proceed_pages()
tonkosti.to_csv('minvod.csv')

Сбор комментариев


  0%|          | 0/12 [00:00<?, ?it/s]

Сбор больших комментариев


  0%|          | 0/44 [00:00<?, ?it/s]

Done. Собрано 83 комментария


In [7]:
class TonkostiCraulerForVocab(BaseCrauler):
    def __init__(self, url, max_pages):
        super().__init__(base_url='https://tonkosti.ru',
                         url=url,
                         max_pages=max_pages)
    
    def parseComments(self, is_exact=False):
        try:
            parent = 'div' if is_exact else 'li'
            for card in self.soup.find_all(parent, class_='reviews__item'):
                    review_text_el = card.find('div', class_="reviews__texts")
                    
                    date_el = card.find('div', class_='reviews__date')
                    date = date_el.text
                    date_el.decompose()
                    
                    review_text = review_text_el.text
                    
                    if 'Читать отзыв целиком' in review_text:
                        try:
                            self.queue.append(card.find('div', class_='reviews__link-to-review-page').find('a').attrs['href'])
                        except:
                            print('Не смог найти ссылку')
                    else:
                        self.result.append({'review_text': review_text, 'date': date})
                        
        except Exception as e:
            print("Error on card", e)

    def _proceed_queue(self):
        for url in tqdm(self.queue):
            self.getSubPage(url)
            self.getSoup()
            self.parseComments(is_exact=True)
    
    def proceed_pages(self, pages=None):
        if pages is None: pages = self.max_pages
        
        print('Сбор комментариев')
        for page_number in tqdm(range(1, pages+1)):
            self.getPage(page_number)
            self.getSoup()
            self.parseComments()
        
        print("Сбор больших комментариев")
        self._proceed_queue()
        
        print(f'Done. Собрано {len(self.result)} комментария')

In [12]:
tonkosti = TonkostiCraulerForVocab(max_pages=186, url='https://tonkosti.ru/%D0%9E%D1%82%D0%B7%D1%8B%D0%B2%D1%8B_%D0%BE%D0%B1_%D0%90%D0%B1%D1%85%D0%B0%D0%B7%D0%B8%D0%B8?page={page}')
tonkosti.proceed_pages()
tonkosti.to_csv('data/vocab_data/abhazia.csv')

Сбор комментариев


  0%|          | 0/186 [00:00<?, ?it/s]

Сбор больших комментариев


  0%|          | 0/656 [00:00<?, ?it/s]

Done. Собрано 1856 комментария


In [15]:
tonkosti = TonkostiCrauler(max_pages=196, url='https://tonkosti.ru/%D0%9E%D1%82%D0%B7%D1%8B%D0%B2%D1%8B_%D0%BE_%D0%9A%D1%80%D0%B0%D1%81%D0%BD%D0%BE%D0%B4%D0%B0%D1%80%D1%81%D0%BA%D0%BE%D0%BC_%D0%BA%D1%80%D0%B0%D0%B5?page={page}')
tonkosti.proceed_pages()
tonkosti.to_csv('data/krasnodarskiy_kray.csv')

Сбор комментариев


  0%|          | 0/196 [00:00<?, ?it/s]

Сбор больших комментариев


  0%|          | 0/631 [00:00<?, ?it/s]

Done. Собрано 1617 комментария
